In [1]:
# import dependencies
import requests
import pandas as pd

In [2]:
# read disappearance.csv as dataframe
import_consumption_path = "datasets/disappearance.csv"

import_consumption_data = pd.read_csv(import_consumption_path)
import_consumption_data

,disappearance,1990,1991,1992,1993,1994,1995,1996,1997,1998,...,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018
0,Austria,1340.123985,1302.085970,1212.333964,1335.000982,1086.218996,1011.769996,1075.788000,1087.079597,1104.738197,...,885.715848,902.854329,1117.345643,1269.019458,1249.0,NaN,NaN,NaN,NaN,NaN
1,Belgium,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,934.292881,870.665179,934.468749,914.636559,1245.0,NaN,NaN,NaN,NaN,NaN
2,Belgium/Luxembourg,1124.109984,708.156980,696.372979,789.436984,958.214990,1073.014984,1038.637071,886.263764,1309.406477,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Bulgaria,101.782998,16.761001,174.878012,326.955006,352.764006,393.408007,246.474995,274.918003,324.160004,...,409.275903,395.357991,359.519751,375.941251,419.0,NaN,NaN,NaN,NaN,NaN
4,Croatia,NaN,NaN,134.545004,131.890000,163.862001,288.481005,304.468998,361.396003,319.730000,...,365.637529,369.838593,367.153143,360.364120,387.0,NaN,NaN,NaN,NaN,NaN
5,Cyprus,30.570999,44.697000,39.303999,77.516000,45.369001,42.678000,51.740999,46.495999,46.205999,...,75.437867,73.905648,80.793525,85.398320,89.0,NaN,NaN,NaN,NaN,NaN
6,Czech Republic,659.433000,525.274000,472.156004,448.395004,508.724000,511.453998,474.992003,484.579999,523.407997,...,525.168046,469.731678,572.458470,685.226286,637.0,NaN,NaN,NaN,NaN,NaN
7,Denmark,865.138999,908.542001,958.834999,890.555997,908.483001,757.796998,895.013997,808.457194,853.660094,...,676.252995,806.488750,762.258628,799.824326,822.0,NaN,NaN,NaN,NaN,NaN
8,Estonia,NaN,NaN,12.909000,44.188998,63.163995,82.848004,84.214010,98.861998,89.530996,...,123.493948,104.782704,66.614409,103.980513,120.0,NaN,NaN,NaN,NaN,NaN
9,Finland,1070.434998,965.553999,1030.327999,1128.863992,1049.507994,734.677996,900.641895,939.425296,1001.365097,...,1058.406243,1080.424215,1092.977075,1082.025985,1095.0,NaN,NaN,NaN,NaN,NaN


In [3]:
# rename disappearance to 'import' and 'disappearance' column to 'country'
import_renamed = import_consumption_data.rename(columns ={"disappearance" : "Country"})
import_renamed.head()

,Country,1990,1991,1992,1993,1994,1995,1996,1997,1998,...,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018
0,Austria,1340.123985,1302.085970,1212.333964,1335.000982,1086.218996,1011.769996,1075.788000,1087.079597,1104.738197,...,885.715848,902.854329,1117.345643,1269.019458,1249.0,NaN,NaN,NaN,NaN,NaN
1,Belgium,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,934.292881,870.665179,934.468749,914.636559,1245.0,NaN,NaN,NaN,NaN,NaN
2,Belgium/Luxembourg,1124.109984,708.156980,696.372979,789.436984,958.214990,1073.014984,1038.637071,886.263764,1309.406477,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Bulgaria,101.782998,16.761001,174.878012,326.955006,352.764006,393.408007,246.474995,274.918003,324.160004,...,409.275903,395.357991,359.519751,375.941251,419.0,NaN,NaN,NaN,NaN,NaN
4,Croatia,NaN,NaN,134.545004,131.890000,163.862001,288.481005,304.468998,361.396003,319.730000,...,365.637529,369.838593,367.153143,360.364120,387.0,NaN,NaN,NaN,NaN,NaN


In [4]:
# fill null values with 0
cleaned_import = import_renamed.fillna(0)
cleaned_import

,Country,1990,1991,1992,1993,1994,1995,1996,1997,1998,...,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018
0,Austria,1340.123985,1302.085970,1212.333964,1335.000982,1086.218996,1011.769996,1075.788000,1087.079597,1104.738197,...,885.715848,902.854329,1117.345643,1269.019458,1249.0,0.0,0.0,0.0,0.0,0.000000
1,Belgium,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,934.292881,870.665179,934.468749,914.636559,1245.0,0.0,0.0,0.0,0.0,0.000000
2,Belgium/Luxembourg,1124.109984,708.156980,696.372979,789.436984,958.214990,1073.014984,1038.637071,886.263764,1309.406477,...,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000
3,Bulgaria,101.782998,16.761001,174.878012,326.955006,352.764006,393.408007,246.474995,274.918003,324.160004,...,409.275903,395.357991,359.519751,375.941251,419.0,0.0,0.0,0.0,0.0,0.000000
4,Croatia,0.000000,0.000000,134.545004,131.890000,163.862001,288.481005,304.468998,361.396003,319.730000,...,365.637529,369.838593,367.153143,360.364120,387.0,0.0,0.0,0.0,0.0,0.000000
5,Cyprus,30.570999,44.697000,39.303999,77.516000,45.369001,42.678000,51.740999,46.495999,46.205999,...,75.437867,73.905648,80.793525,85.398320,89.0,0.0,0.0,0.0,0.0,0.000000
6,Czech Republic,659.433000,525.274000,472.156004,448.395004,508.724000,511.453998,474.992003,484.579999,523.407997,...,525.168046,469.731678,572.458470,685.226286,637.0,0.0,0.0,0.0,0.0,0.000000
7,Denmark,865.138999,908.542001,958.834999,890.555997,908.483001,757.796998,895.013997,808.457194,853.660094,...,676.252995,806.488750,762.258628,799.824326,822.0,0.0,0.0,0.0,0.0,0.000000
8,Estonia,0.000000,0.000000,12.909000,44.188998,63.163995,82.848004,84.214010,98.861998,89.530996,...,123.493948,104.782704,66.614409,103.980513,120.0,0.0,0.0,0.0,0.0,0.000000
9,Finland,1070.434998,965.553999,1030.327999,1128.863992,1049.507994,734.677996,900.641895,939.425296,1001.365097,...,1058.406243,1080.424215,1092.977075,1082.025985,1095.0,0.0,0.0,0.0,0.0,0.000000


In [5]:
# set path for exporting country consumption and reading it in dataframe
export_consumption_path = "datasets/domestic-consumption.csv"

export_consumption_data = pd.read_csv(export_consumption_path)
export_consumption_data

,domestic_consumption,1990,1991,1992,1993,1994,1995,1996,1997,1998,...,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018
0,Angola,20.000,30.000,35.000,20.000,25.000,10.000,20.000,40.000,30.000,...,30.000,30.000,30.000,30.000,30.000,30.000,30.000,30.0000,30.000,30.000
1,Bolivia (Plurinational State of),25.000,27.000,27.500,28.500,29.500,30.500,31.500,32.500,33.000,...,46.000,47.500,49.000,50.500,52.000,53.500,55.000,57.0000,58.500,60.000
2,Brazil,8200.000,8500.000,8900.000,9100.000,9300.000,10100.000,11000.000,11500.000,12200.000,...,18390.000,19132.000,19720.000,20330.000,20085.000,20333.000,20508.000,21225.0000,21997.000,22250.000
3,Burundi,2.000,1.600,1.700,1.910,2.000,2.000,2.000,2.000,2.000,...,1.399,2.000,2.000,2.000,2.000,2.000,2.000,2.0000,2.000,2.000
4,Ecuador,350.000,350.000,350.000,350.000,350.000,350.000,300.000,300.000,300.000,...,150.000,150.000,150.000,150.000,155.000,155.000,155.000,155.0000,155.000,155.000
5,Indonesia,1242.000,1280.000,1319.000,1359.000,1400.000,1443.000,1486.000,1532.000,1578.000,...,3333.000,3333.000,3667.000,3900.000,4250.000,4417.000,4550.000,4650.0000,4750.000,4800.000
6,Madagascar,350.000,300.000,350.000,360.000,166.666,167.000,167.000,167.000,167.000,...,467.000,467.000,450.000,430.000,410.000,390.000,370.000,360.0000,365.000,375.000
7,Malawi,2.000,2.000,2.000,2.000,2.000,2.000,2.000,2.000,2.000,...,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.0000,1.000,1.000
8,Papua New Guinea,3.000,3.000,3.000,2.000,2.000,2.000,2.000,2.000,1.000,...,2.000,1.430,1.236,1.711,2.039,1.924,1.800,2.0000,2.000,2.000
9,Paraguay,15.000,15.000,15.000,20.000,20.000,20.000,20.000,20.000,20.000,...,20.000,20.000,20.000,20.000,20.000,20.000,20.000,20.0000,20.000,20.000


In [6]:
# rename 'domestic_consumption' to 'country'
export_renamed = export_consumption_data.rename(columns = {'domestic_consumption' : 'Country'})
export_renamed

,Country,1990,1991,1992,1993,1994,1995,1996,1997,1998,...,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018
0,Angola,20.000,30.000,35.000,20.000,25.000,10.000,20.000,40.000,30.000,...,30.000,30.000,30.000,30.000,30.000,30.000,30.000,30.0000,30.000,30.000
1,Bolivia (Plurinational State of),25.000,27.000,27.500,28.500,29.500,30.500,31.500,32.500,33.000,...,46.000,47.500,49.000,50.500,52.000,53.500,55.000,57.0000,58.500,60.000
2,Brazil,8200.000,8500.000,8900.000,9100.000,9300.000,10100.000,11000.000,11500.000,12200.000,...,18390.000,19132.000,19720.000,20330.000,20085.000,20333.000,20508.000,21225.0000,21997.000,22250.000
3,Burundi,2.000,1.600,1.700,1.910,2.000,2.000,2.000,2.000,2.000,...,1.399,2.000,2.000,2.000,2.000,2.000,2.000,2.0000,2.000,2.000
4,Ecuador,350.000,350.000,350.000,350.000,350.000,350.000,300.000,300.000,300.000,...,150.000,150.000,150.000,150.000,155.000,155.000,155.000,155.0000,155.000,155.000
5,Indonesia,1242.000,1280.000,1319.000,1359.000,1400.000,1443.000,1486.000,1532.000,1578.000,...,3333.000,3333.000,3667.000,3900.000,4250.000,4417.000,4550.000,4650.0000,4750.000,4800.000
6,Madagascar,350.000,300.000,350.000,360.000,166.666,167.000,167.000,167.000,167.000,...,467.000,467.000,450.000,430.000,410.000,390.000,370.000,360.0000,365.000,375.000
7,Malawi,2.000,2.000,2.000,2.000,2.000,2.000,2.000,2.000,2.000,...,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.0000,1.000,1.000
8,Papua New Guinea,3.000,3.000,3.000,2.000,2.000,2.000,2.000,2.000,1.000,...,2.000,1.430,1.236,1.711,2.039,1.924,1.800,2.0000,2.000,2.000
9,Paraguay,15.000,15.000,15.000,20.000,20.000,20.000,20.000,20.000,20.000,...,20.000,20.000,20.000,20.000,20.000,20.000,20.000,20.0000,20.000,20.000


In [7]:
# merge import and export consumption dataframes on 'country' in 'concat_consumption'
frames = [cleaned_import, export_renamed]

concat_consumption = pd.concat(frames)
concat_consumption = concat_consumption.set_index('Country')
concat_consumption = concat_consumption.reset_index()

concat_consumption

,Country,1990,1991,1992,1993,1994,1995,1996,1997,1998,...,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018
0,Austria,1340.123985,1302.085970,1212.333964,1335.000982,1086.218996,1011.769996,1075.788000,1087.079597,1104.738197,...,885.715848,902.854329,1117.345643,1269.019458,1249.0,0.0,0.0,0.0,0.0,0.0
1,Belgium,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,934.292881,870.665179,934.468749,914.636559,1245.0,0.0,0.0,0.0,0.0,0.0
2,Belgium/Luxembourg,1124.109984,708.156980,696.372979,789.436984,958.214990,1073.014984,1038.637071,886.263764,1309.406477,...,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
3,Bulgaria,101.782998,16.761001,174.878012,326.955006,352.764006,393.408007,246.474995,274.918003,324.160004,...,409.275903,395.357991,359.519751,375.941251,419.0,0.0,0.0,0.0,0.0,0.0
4,Croatia,0.000000,0.000000,134.545004,131.890000,163.862001,288.481005,304.468998,361.396003,319.730000,...,365.637529,369.838593,367.153143,360.364120,387.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
86,Trinidad & Tobago,8.000000,11.500000,10.000000,10.000000,14.000000,14.000000,14.000000,14.000000,14.000000,...,11.000000,10.000000,10.000000,10.000000,10.0,10.0,10.0,10.0,10.0,10.0
87,Uganda,70.000000,75.000000,75.000000,75.000000,80.000000,80.000000,80.000000,101.190000,116.000000,...,188.700000,204.000000,210.000000,216.000000,220.8,229.2,234.0,240.0,244.8,250.0
88,Venezuela,782.390000,815.480000,849.970000,885.930000,923.400000,962.460000,1003.180000,1045.610000,1089.840000,...,1650.000000,1650.000000,1650.000000,1650.000000,1650.0,1650.0,1650.0,1650.0,1600.0,1550.0
89,Viet Nam,150.000000,230.000000,250.000000,267.000000,267.000000,281.845000,297.516000,314.058000,331.519000,...,1208.000000,1583.000000,1650.000000,1825.000000,2000.0,2200.0,2300.0,2400.0,2500.0,2700.0


In [8]:
# drop 'Belgium/Luxembourg'
concat_consumption.drop(axis= 0,labels=[2], inplace=True)

In [9]:
# rename a whole bunch of entries with strange blank spaces
concat_consumption['Country'].replace({'Viet Nam' : "Vietnam"}, inplace=True)
concat_consumption['Country'].replace({"Côte d'Ivoire" : "Cote d'Ivoire"}, inplace=True)
concat_consumption['Country'].replace({'Bolivia (Plurinational State of)' : "Bolivia"}, inplace=True)
concat_consumption['Country'].replace({"Lao People's Democratic Republic" : "Lao PDR"}, inplace=True)
concat_consumption['Country'].replace({'   Austria' : "Austria"}, inplace=True)
concat_consumption['Country'].replace({"   Belgium" : "Belgium"}, inplace=True)
concat_consumption['Country'].replace({'   Bulgaria' : "Bulgaria"}, inplace=True)
concat_consumption['Country'].replace({"   Croatia" : "Croatia"}, inplace=True)
concat_consumption['Country'].replace({'   Cyprus' : "Cyprus"}, inplace=True)
concat_consumption['Country'].replace({"   Czech Republic" : "Czech Republic"}, inplace=True)
concat_consumption['Country'].replace({'   Denmark' : "Denmark"}, inplace=True)
concat_consumption['Country'].replace({"   Estonia" : "Estonia"}, inplace=True)
concat_consumption['Country'].replace({'   Finland' : "Finland"}, inplace=True)
concat_consumption['Country'].replace({"   France" : "France"}, inplace=True)
concat_consumption['Country'].replace({'   Germany' : "Germany"}, inplace=True)
concat_consumption['Country'].replace({"   Greece" : "Greece"}, inplace=True)
concat_consumption['Country'].replace({'   Hungary' : "Hungary"}, inplace=True)
concat_consumption['Country'].replace({"   Ireland" : "Ireland"}, inplace=True)
concat_consumption['Country'].replace({'   Italy' : "Italy"}, inplace=True)
concat_consumption['Country'].replace({"   Latvia" : "Latvia"}, inplace=True)
concat_consumption['Country'].replace({'   Lithuania' : "Lithuania"}, inplace=True)
concat_consumption['Country'].replace({"   Luxembourg" : "Luxembourg"}, inplace=True)
concat_consumption['Country'].replace({'   Malta' : "Malta"}, inplace=True)
concat_consumption['Country'].replace({"   Netherlands" : "Netherlands"}, inplace=True)
concat_consumption['Country'].replace({'   Poland' : "Poland"}, inplace=True)
concat_consumption['Country'].replace({"   Portugal" : "Portugal"}, inplace=True)
concat_consumption['Country'].replace({'   Romania' : "Romania"}, inplace=True)
concat_consumption['Country'].replace({"   Slovakia" : "Slovakia"}, inplace=True)
concat_consumption['Country'].replace({'   Slovenia' : "Slovenia"}, inplace=True)
concat_consumption['Country'].replace({"   Spain" : "Spain"}, inplace=True)
concat_consumption['Country'].replace({'   Sweden' : "Sweden"}, inplace=True)
concat_consumption['Country'].replace({"   United Kingdom" : "United Kingdom"}, inplace=True)


In [10]:
# set path for gdp_per_capita.csv and open it in a dataframe
gdp_path = "datasets/gdp_per_capita.csv"
gdp_data = pd.read_csv(gdp_path)

gdp_data

,Country Name,Code,1960,1961,1962,1963,1964,1965,1966,1967,...,2012,2013,2014,2015,2016,2017,2018,2019,2020,Unnamed: 65
0,Aruba,ABW,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,24712.493263,26441.619936,26893.011506,28396.908423,28452.170615,29350.805019,30253.279358,NaN,NaN,NaN
1,Africa Eastern and Southern,AFE,147.612227,147.014904,156.189192,182.243917,162.347592,180.214908,190.845484,192.337167,...,1736.166560,1713.899299,1703.596298,1549.037940,1431.778723,1573.063386,1574.978648,1530.059177,1359.618224,NaN
2,Afghanistan,AFG,59.773234,59.860900,58.458009,78.706429,82.095307,101.108325,137.594298,160.898434,...,638.845852,624.315455,614.223342,556.007221,512.012778,516.679862,485.668419,494.179350,516.747871,NaN
3,Africa Western and Central,AFW,107.932233,113.081647,118.831107,123.442888,131.854402,138.526332,144.326212,128.582470,...,1965.118485,2157.481149,2212.853135,1894.310195,1673.835527,1613.473553,1704.139603,1777.918672,1710.073363,NaN
4,Angola,AGO,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,5100.097027,5254.881126,5408.411700,4166.979833,3506.073128,4095.810057,3289.643995,2809.626088,1776.166868,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
261,Kosovo,XKX,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,3410.859780,3704.784221,3902.676013,3520.766449,3759.560246,4009.380987,4384.048892,4416.108358,4346.637931,NaN
262,"Yemen, Rep.",YEM,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1446.536472,1607.152173,1674.002572,1601.830063,1152.720966,964.264811,758.145242,NaN,NaN,NaN
263,South Africa,ZAF,443.009920,454.962013,473.011405,511.497364,548.996058,584.704163,621.220543,675.135345,...,8222.197279,7467.079185,6988.808739,6259.839681,5756.965741,6690.939847,7005.095413,6624.761865,5655.867654,NaN
264,Zambia,ZMB,232.188564,220.042067,212.578449,213.896759,242.384473,303.281740,343.373670,360.201239,...,1763.069442,1878.346811,1762.427817,1338.290927,1280.806543,1535.196574,1516.368371,1305.001031,985.132436,NaN


In [11]:
# rename column 'country name' to 'country'
gdp_renamed = gdp_data.rename(columns={"Country Name" : "Country"})
gdp_renamed

,Country,Code,1960,1961,1962,1963,1964,1965,1966,1967,...,2012,2013,2014,2015,2016,2017,2018,2019,2020,Unnamed: 65
0,Aruba,ABW,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,24712.493263,26441.619936,26893.011506,28396.908423,28452.170615,29350.805019,30253.279358,NaN,NaN,NaN
1,Africa Eastern and Southern,AFE,147.612227,147.014904,156.189192,182.243917,162.347592,180.214908,190.845484,192.337167,...,1736.166560,1713.899299,1703.596298,1549.037940,1431.778723,1573.063386,1574.978648,1530.059177,1359.618224,NaN
2,Afghanistan,AFG,59.773234,59.860900,58.458009,78.706429,82.095307,101.108325,137.594298,160.898434,...,638.845852,624.315455,614.223342,556.007221,512.012778,516.679862,485.668419,494.179350,516.747871,NaN
3,Africa Western and Central,AFW,107.932233,113.081647,118.831107,123.442888,131.854402,138.526332,144.326212,128.582470,...,1965.118485,2157.481149,2212.853135,1894.310195,1673.835527,1613.473553,1704.139603,1777.918672,1710.073363,NaN
4,Angola,AGO,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,5100.097027,5254.881126,5408.411700,4166.979833,3506.073128,4095.810057,3289.643995,2809.626088,1776.166868,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
261,Kosovo,XKX,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,3410.859780,3704.784221,3902.676013,3520.766449,3759.560246,4009.380987,4384.048892,4416.108358,4346.637931,NaN
262,"Yemen, Rep.",YEM,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1446.536472,1607.152173,1674.002572,1601.830063,1152.720966,964.264811,758.145242,NaN,NaN,NaN
263,South Africa,ZAF,443.009920,454.962013,473.011405,511.497364,548.996058,584.704163,621.220543,675.135345,...,8222.197279,7467.079185,6988.808739,6259.839681,5756.965741,6690.939847,7005.095413,6624.761865,5655.867654,NaN
264,Zambia,ZMB,232.188564,220.042067,212.578449,213.896759,242.384473,303.281740,343.373670,360.201239,...,1763.069442,1878.346811,1762.427817,1338.290927,1280.806543,1535.196574,1516.368371,1305.001031,985.132436,NaN


In [12]:
# replace name of country 'United States' with 'USA'
gdp_renamed['Country'].replace({'United States':'USA'}, inplace=True)

In [13]:
# drop unnecessary years
gdp_renamed.drop(['Code','1960', '1961', '1962', '1963', '1964', '1965', '1966', '1967', '1968', '1969','1970', '1971', '1972', '1973', '1974', '1975', '1976', '1977', '1978', '1979', '1980', '1981', '1982', '1983', '1984', '1985', '1986', '1987', '1988', '1989', '2019', '2020', 'Unnamed: 65'], axis=1, inplace=True)
gdp_renamed

,Country,1990,1991,1992,1993,1994,1995,1996,1997,1998,...,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018
0,Aruba,12306.717679,13495.794301,14045.474566,14938.681573,16239.987852,16439.356361,16583.875856,17927.959439,19077.687427,...,24631.182052,23513.527697,24985.013919,24712.493263,26441.619936,26893.011506,28396.908423,28452.170615,29350.805019,30253.279358
1,Africa Eastern and Southern,696.988145,704.427088,683.336134,706.488039,702.415966,771.410097,747.841550,765.967485,703.091356,...,1412.111284,1634.448253,1770.736230,1736.166560,1713.899299,1703.596298,1549.037940,1431.778723,1573.063386,1574.978648
2,Afghanistan,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,437.268740,543.306526,591.190030,638.845852,624.315455,614.223342,556.007221,512.012778,516.679862,485.668419
3,Africa Western and Central,594.726009,558.435681,547.663093,445.656206,378.940493,462.864809,523.763527,515.234581,513.642551,...,1486.572654,1687.551979,1862.308267,1965.118485,2157.481149,2212.853135,1894.310195,1673.835527,1613.473553,1704.139603
4,Angola,948.338178,NaN,NaN,NaN,251.078348,398.790998,522.679624,514.385174,423.619049,...,3122.781599,3587.883645,4615.468219,5100.097027,5254.881126,5408.411700,4166.979833,3506.073128,4095.810057,3289.643995
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
261,Kosovo,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,2847.555339,3009.559334,3540.891789,3410.859780,3704.784221,3902.676013,3520.766449,3759.560246,4009.380987,4384.048892
262,"Yemen, Rep.",482.248121,482.060571,498.963382,393.739059,291.472071,285.569593,374.011431,428.060596,384.499940,...,1116.084396,1334.784902,1374.621401,1446.536472,1607.152173,1674.002572,1601.830063,1152.720966,964.264811,758.145242
263,South Africa,3139.966225,3285.972326,3479.083205,3713.921206,3784.449144,4144.613723,3864.415112,3930.836564,3502.166447,...,6532.736968,8148.961202,8810.930651,8222.197279,7467.079185,6988.808739,6259.839681,5756.965741,6690.939847,7005.095413
264,Zambia,408.769331,409.727275,376.498819,378.125559,412.260752,418.515025,385.152152,448.370160,358.555959,...,1159.907499,1489.459087,1672.907535,1763.069442,1878.346811,1762.427817,1338.290927,1280.806543,1535.196574,1516.368371


In [14]:
# renaming more strangely named countries
gdp_renamed['Country'].replace({'Trinidad and Tobago' : "Trinidad & Tobago"}, inplace=True)
gdp_renamed['Country'].replace({"Venezuela, RB" : "Venezuela"}, inplace=True)
gdp_renamed['Country'].replace({'Slovak Republic' : "Slovakia"}, inplace=True)
gdp_renamed['Country'].replace({"Congo, Rep." : "Congo"}, inplace=True)
gdp_renamed['Country'].replace({'Congo, Dem. Rep.' : "Democratic Republic of Congo"}, inplace=True)
gdp_renamed['Country'].replace({"Yemen, Rep." : "Yemen"}, inplace=True)


In [15]:
# dropping all entries for countries not found in the 'concat_consumption' dataframe
gdp_trimmed = gdp_renamed[gdp_renamed.Country.isin(concat_consumption.Country)]
gdp_trimmed

,Country,1990,1991,1992,1993,1994,1995,1996,1997,1998,...,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018
4,Angola,948.338178,NaN,NaN,NaN,251.078348,398.790998,522.679624,514.385174,423.619049,...,3122.781599,3587.883645,4615.468219,5100.097027,5254.881126,5408.411700,4166.979833,3506.073128,4095.810057,3289.643995
14,Austria,21680.989623,22410.911767,24880.164118,24081.527793,25646.700659,30325.849582,29809.076773,26705.478599,27361.875111,...,47963.179402,46858.043273,51374.958407,48567.695286,50716.708706,51717.495941,44178.047378,45276.831435,47312.006233,51461.433215
16,Burundi,208.146679,209.778011,190.488880,161.887525,156.812381,167.098888,143.402324,158.914670,144.493017,...,212.137057,234.235539,249.577979,252.358871,256.975653,274.857836,305.549653,260.565221,253.826354,238.783467
17,Belgium,20600.375279,21041.660652,23372.619171,22283.936021,24208.554793,28413.826439,27489.555177,24820.938050,25338.443293,...,44583.544807,44141.878142,47348.525020,44673.115876,46744.662544,47700.540360,40991.808138,41984.103034,44089.310088,47519.553097
18,Benin,393.686768,385.753841,317.962736,411.926180,279.666749,367.387714,387.432098,361.099982,379.441768,...,1088.757906,1036.534515,1130.273251,1145.140105,1251.209767,1291.410185,1076.796698,1087.287331,1136.593872,1241.825298
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
254,Venezuela,2475.380473,2661.033851,2938.160197,2857.938660,2720.367063,3529.589128,3151.269278,3758.844934,3921.718474,...,11765.099757,13825.357116,10955.538014,12986.222692,12457.768602,16055.645317,NaN,NaN,NaN,NaN
257,Vietnam,95.188260,138.447450,139.200123,182.308402,221.129175,276.812663,324.147224,348.017407,348.324317,...,1217.268583,1317.890646,1525.119025,1735.152116,1886.690170,2030.278447,2085.101349,2192.174482,2365.521615,2566.447487
262,Yemen,482.248121,482.060571,498.963382,393.739059,291.472071,285.569593,374.011431,428.060596,384.499940,...,1116.084396,1334.784902,1374.621401,1446.536472,1607.152173,1674.002572,1601.830063,1152.720966,964.264811,758.145242
264,Zambia,408.769331,409.727275,376.498819,378.125559,412.260752,418.515025,385.152152,448.370160,358.555959,...,1159.907499,1489.459087,1672.907535,1763.069442,1878.346811,1762.427817,1338.290927,1280.806543,1535.196574,1516.368371


In [16]:
# save cleaned datasets to 'cleaned-datasets' directory
gdp_trimmed.to_csv('cleaned-datasets/trimmed-gdp.csv', index=False)
concat_consumption.to_csv('cleaned-datasets/concat-consumption.csv', index=False)